In [1]:
import numpy as np
from scipy import stats
from scipy.stats import entropy
from pywt import wavedec
from scipy.signal import welch


from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from mne_features.univariate import compute_spect_entropy, compute_wavelet_coef_energy
# https://mne.tools/mne-features/api.html

import mne
from mne.io import concatenate_raws, read_raw_edf
import glob


from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, ShuffleSplit, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from mne.decoding import CSP, SPoC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from mne.decoding import (
    SlidingEstimator,
    GeneralizingEstimator,
    Scaler,
    cross_val_multiscore,
    LinearModel,
    get_coef,
    Vectorizer,
    CSP,
)
import numpy as np
from mne.preprocessing import ICA

from lightgbm import LGBMClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from utils import preprocess_data


import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Flatten, BatchNormalization, Conv2D, DepthwiseConv2D, AveragePooling2D, Activation, SeparableConv2D, SpatialDropout1D
from tensorflow.keras.utils import to_categorical
import mne
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
import glob
import numpy as np
from utils import preprocess_data
from mne.preprocessing import ICA
from sklearn.decomposition import FastICA

2023-09-06 22:48:08.876950: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


https://arxiv.org/ftp/arxiv/papers/1312/1312.2877.pdf

In [2]:
files = glob.glob('../../files/S001/*.edf')

'''
https://github.com/mne-tools/mne-python/blob/main/mne/datasets/eegbci/eegbci.py#L110
=========  ===================================
run        task
=========  ===================================
1          Baseline, eyes open
2          Baseline, eyes closed
3, 7, 11   Motor execution: left vs right hand
4, 8, 12   Motor imagery: left vs right hand
5, 9, 13   Motor execution: hands vs feet
6, 10, 14  Motor imagery: hands vs feet
=========  ===================================
'''
raws = []
f = [5,9,13]
# f = [6,10,14]
for i in f:
    print(i)
    current_file = files[i-1]
    r = read_raw_edf(current_file, preload=True, stim_channel='auto')
    events, _ = mne.events_from_annotations(r)
    if i in [5, 9, 13]:
        new_labels_events = {1:'rest', 2:'action_hand', 3:'action_feet'} # action
    else:
        new_labels_events = {1:'rest', 2:'imagine_hand', 3:'imagine_feet'} # imagine
    new_annot = mne.annotations_from_events(events=events, event_desc=new_labels_events, sfreq=r.info['sfreq'], orig_time=r.info['meas_date'])
    r.set_annotations(new_annot)
    raws.append(r)
    
raw_obj = concatenate_raws(raws)

original_raw = raw_obj.copy()
original_raw.info

5
Extracting EDF parameters from /Users/owalid/42/post_intership/total-perspective-vortex/files/S001/S001R11.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
9
Extracting EDF parameters from /Users/owalid/42/post_intership/total-perspective-vortex/files/S001/S001R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
13
Extracting EDF parameters from /Users/owalid/42/post_intership/total-perspective-vortex/files/S001/S001R02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Used Annotations descriptions: ['T0']


<Info | 7 non-empty values
 bads: []
 ch_names: Fc5., Fc3., Fc1., Fcz., Fc2., Fc4., Fc6., C5.., C3.., C1.., ...
 chs: 64 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 80.0 Hz
 meas_date: 2009-08-12 16:15:00 UTC
 nchan: 64
 projs: []
 sfreq: 160.0 Hz
>

In [3]:
notch_freq = 50
low_cutoff = 8
high_cutoff = 30

raw_obj.notch_filter(notch_freq, fir_design='firwin')
raw_obj.filter(low_cutoff, high_cutoff, fir_design='firwin')

Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1057 samples (6.606 sec)

Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth:

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.1s finished


<RawEDF | S001R11.edf, 64 x 49760 (311.0 s), ~24.4 MB, data loaded>

In [4]:
event_id = {'action_hand': 1, 'action_feet': 2}
events, event_dict = mne.events_from_annotations(original_raw, event_id=event_id)

# Epoch your data for different analyses
tmin_erd = -2
tmax_erd = 0
tmin_ers = 4.1
tmax_ers = 5.1
tmin_mrcp = -2
tmax_mrcp = 0

raw_obj_erd = raw_obj.copy()
raw_obj_erd.filter(8, 30, fir_design='firwin')
raw_obj_ers = raw_obj.copy()
raw_obj_ers.filter(8, 30, fir_design='firwin')
raw_obj_mrcp = raw_obj.copy()
raw_obj_mrcp.filter(None, 3, fir_design='firwin')

erd_epochs = mne.Epochs(raw_obj, events, event_id=event_id, tmin=tmin_erd, tmax=tmax_erd, baseline=None)
ers_epochs = mne.Epochs(raw_obj, events, event_id=event_id, tmin=tmin_ers, tmax=tmax_ers, baseline=None)
mrcp_epochs = mne.Epochs(raw_obj, events, event_id=event_id, tmin=tmin_mrcp, tmax=tmax_mrcp, baseline=None)

Used Annotations descriptions: ['action_feet', 'action_hand']
Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 265 samples (1.656 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.1s finished


Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 265 samples (1.656 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.1s finished


Filtering raw data in 3 contiguous segments
Setting up low-pass filter at 3 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 3.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 4.00 Hz)
- Filter length: 265 samples (1.656 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.1s finished


Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated


In [5]:
erd_epochs.get_data().shape

Using data from preloaded Raw for 30 events and 321 original time points ...
0 bad epochs dropped


(30, 64, 321)

In [6]:
def get_features_from_epoch_one(epoch_data, ica):
    current_data = epoch_data
    activation_vector = ica.fit(current_data)
    activation_vector = activation_vector.components_
    mean_current_data = np.mean(current_data, axis=0)
    activation_vector -= mean_current_data
    mean_activation_vector = np.mean(activation_vector, axis=1)
    std_activation_vector = np.std(activation_vector, axis=1)
    power_activation_vector = np.mean(np.abs(activation_vector) ** 2, axis=1)
    energy_activation_vector = np.sum(np.abs(activation_vector) ** 2, axis=1)
    variance_activation_vector = np.var(activation_vector, axis=1)
    rms_activation_vector = np.sqrt(np.mean(activation_vector ** 2, axis=1))

    epoch_features = np.hstack((mean_activation_vector, std_activation_vector, energy_activation_vector, power_activation_vector, variance_activation_vector, rms_activation_vector))
    return epoch_features

In [7]:
n_components = None
feature_matrix = []
labels = []
ica = FastICA(n_components=n_components, random_state=97, max_iter=800)

for ii in range(erd_epochs.get_data().shape[0]):
    current_data_erd = erd_epochs.get_data()[ii]
    features_erd = get_features_from_epoch_one(current_data_erd, ica)
    feature_matrix.append(features_erd)

yy_erd = erd_epochs.events[:, -1] - 1

for ii in range(ers_epochs.get_data().shape[0]):
    current_data_ers = ers_epochs.get_data()[ii]
    features_ers = get_features_from_epoch_one(current_data_ers, ica)
    feature_matrix.append(features_ers)
yy_ers = ers_epochs.events[:, -1] - 1

for ii in range(mrcp_epochs.get_data().shape[0]):
    current_data_mrcp = mrcp_epochs.get_data()[ii]
    features_mrcp = get_features_from_epoch_one(current_data_mrcp, ica)
    feature_matrix.append(features_mrcp)
yy_mrcp = mrcp_epochs.events[:, -1] - 1

y = np.hstack((yy_erd, yy_ers, yy_mrcp))
print(y.shape)
feature_matrix = np.array(feature_matrix)
print(feature_matrix.shape)

Using data from preloaded Raw for 30 events and 321 original time points ...
Using data from preloaded Raw for 30 events and 321 original time points ...
Using data from preloaded Raw for 30 events and 321 original time points ...
Using data from preloaded Raw for 30 events and 321 original time points ...
Using data from preloaded Raw for 30 events and 321 original time points ...
Using data from preloaded Raw for 30 events and 321 original time points ...
Using data from preloaded Raw for 30 events and 321 original time points ...
Using data from preloaded Raw for 30 events and 321 original time points ...
Using data from preloaded Raw for 30 events and 321 original time points ...
Using data from preloaded Raw for 30 events and 321 original time points ...
Using data from preloaded Raw for 30 events and 321 original time points ...
Using data from preloaded Raw for 30 events and 321 original time points ...
Using data from preloaded Raw for 30 events and 321 original time points ...

In [8]:
labels = y.copy()
features = feature_matrix.copy()

In [9]:
labels.shape, feature_matrix.shape

((88,), (88, 384))

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feature_matrix, labels, test_size=0.2, random_state=42)


In [11]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Create and train the Naive Bayes classifier
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)

# Predict labels on the test set
nb_predictions = nb_classifier.predict(X_test)

# Calculate accuracy
nb_accuracy = accuracy_score(y_test, nb_predictions)
print(f'Naive Bayes Accuracy: {nb_accuracy * 100:.2f}%')


Naive Bayes Accuracy: 38.89%


In [12]:
from sklearn.neural_network import MLPClassifier

# Create and train the ANN classifier
ann_classifier = MLPClassifier(hidden_layer_sizes=(5), max_iter=5800, activation='relu', random_state=42)
ann_classifier.fit(X_train, y_train)

# Predict labels on the test set
ann_predictions = ann_classifier.predict(X_test)

# Calculate accuracy
ann_accuracy = accuracy_score(y_test, ann_predictions)
print(f'ANN Accuracy: {ann_accuracy * 100:.2f}%')


ANN Accuracy: 38.89%


In [13]:
shuffle_split = ShuffleSplit(n_splits=10, test_size=0.3, random_state=42)

models = [
    # ('Gradient Boosting', GradientBoostingClassifier(), {'model__n_estimators': [50, 100]}),
    # ('Linear discriminant analysis', LinearDiscriminantAnalysis(), {'model__solver': ['svd'], 'model__tol': [0.0001, 0.00001]}),   
    # ('Random Forest', RandomForestClassifier(), {'model__n_estimators': [50,100,200,300,400]}),
    # ('SVM', SVC(), {'model__C': [0.5, 1, 3, 5, 10], 'model__gamma': [3, 4, 5, 10], 'model__degree': [1, 3, 5, 8], 'model__kernel': ['linear', 'rbf']}),
    # ('KNN', KNeighborsClassifier(), {'model__n_neighbors': [4,5,6]}),
    ('MLP', MLPClassifier(), {'model__hidden_layer_sizes': [(100, 50), (200, 100)],  'model__max_iter': [5000]}),
    ('Decision Tree', DecisionTreeClassifier(), {'model__max_depth': [10, 15, 20, 30, 50], 'model__min_samples_split': [2, 10, 15], 'model__max_depth': [2, 3, 5, 10, 15]}),
    # ('XGB', XGBClassifier(), {'model__n_estimators': [50, 100, 200], 'model__learning_rate': [0.0001, 0.045, 0.05, 0.001]})
]

pipelines = []
# csp = CSP()
for name, model, param_grid in models:
    pipeline = Pipeline([
        # ('csp', csp),
        ('model', model)
    ])
    # param_grid['csp__n_components'] = [5, 10, 15]
    pipelines.append((name, pipeline, param_grid))


results = []
for name, pipeline, param_grid in pipelines:
    grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=shuffle_split, n_jobs=-1)
    grid_search.fit(features, labels)
    results.append((name, grid_search))


res_grid = []
for name, grid_search in results:
    print(f"Model: {name}")
    print(f"Best Parameters: {grid_search.best_params_}")
    print(f"Best Cross-Validated Accuracy: {grid_search.best_score_:.2f}")
    print("\n")

Model: MLP
Best Parameters: {'model__hidden_layer_sizes': (200, 100), 'model__max_iter': 5000}
Best Cross-Validated Accuracy: 0.54


Model: Decision Tree
Best Parameters: {'model__max_depth': 5, 'model__min_samples_split': 2}
Best Cross-Validated Accuracy: 0.81


